# Imports

In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pyplot as plt # For data viz
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

print('System Version:', sys.version)
print('PyTorch version', torch.__version__)
print('Torchvision version', torchvision.__version__)
print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)

System Version: 3.13.7 | packaged by conda-forge | (main, Sep  3 2025, 14:30:35) [GCC 14.3.0]
PyTorch version 2.6.0
Torchvision version 0.21.0
Numpy version 2.3.3
Pandas version 2.3.2


# Create Pytorch Dataset and Dataloader

In [55]:
# Create the playing card dataset class

class PlayingCardDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)

    def __len__(self): # How many examples are in the dataset?
        return len(self.data)

    def __getitem__(self, idx): # Takes the index location in the dataset and returns one item.
        return self.data[idx]

    @property
    def classes(self):
        return self.data.classes

In [56]:
# Name the data directory

data_dir = '/home/asunkari/neuro-ml/01_learn_pytorch/archive/train'

In [57]:
# Associate the numerical label value with the actual card name using a dictionary

target_to_class = {v: k for k, v in ImageFolder(data_dir).class_to_idx.items()}
print(target_to_class)

{0: 'ace of clubs', 1: 'ace of diamonds', 2: 'ace of hearts', 3: 'ace of spades', 4: 'eight of clubs', 5: 'eight of diamonds', 6: 'eight of hearts', 7: 'eight of spades', 8: 'five of clubs', 9: 'five of diamonds', 10: 'five of hearts', 11: 'five of spades', 12: 'four of clubs', 13: 'four of diamonds', 14: 'four of hearts', 15: 'four of spades', 16: 'jack of clubs', 17: 'jack of diamonds', 18: 'jack of hearts', 19: 'jack of spades', 20: 'joker', 21: 'king of clubs', 22: 'king of diamonds', 23: 'king of hearts', 24: 'king of spades', 25: 'nine of clubs', 26: 'nine of diamonds', 27: 'nine of hearts', 28: 'nine of spades', 29: 'queen of clubs', 30: 'queen of diamonds', 31: 'queen of hearts', 32: 'queen of spades', 33: 'seven of clubs', 34: 'seven of diamonds', 35: 'seven of hearts', 36: 'seven of spades', 37: 'six of clubs', 38: 'six of diamonds', 39: 'six of hearts', 40: 'six of spades', 41: 'ten of clubs', 42: 'ten of diamonds', 43: 'ten of hearts', 44: 'ten of spades', 45: 'three of clu

In [58]:
# Transform all images to the same size

transform = transforms.Compose([
    transforms.Resize((128, 128)),     # resize to 128px by 128px
    transforms.ToTensor(),             # convert the regular image (visual) to a tensor
])

In [59]:
# Import dataset

dataset = PlayingCardDataset(data_dir, transform)

In [60]:
# Dataloader setup

dataloader = DataLoader(dataset, batch_size=32, shuffle=True) # choose batch size and whether to shuffle the data (better for training)

# PyTorch Model

In [61]:
class SimpleCardClassifier(nn.Module):
    def __init__(self, num_classes=53):
        super(SimpleCardClassifier, self).__init__()
        # Where we define all the parts of the model.
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1]) # remove last base layer from that model

        enet_out_size = 1280 # default number of outputs from efficient net b0
        # Make a classifier
        self.classifier = nn.Linear(enet_out_size, num_classes) # convert 1280 outputs into 53 classes

    def forward(self, x):
        # Connect these parts and return the output.
        x = self.features(x)
        output = self.classifier(x)
        return output

In [63]:
model = SimpleCardClassifier(num_classes=53)
model(images).shape

torch.Size([32, 53])